In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy
import math
import pandas_ta as ta
from pathlib import Path
import yfinance as yf
from datetime import datetime, timedelta

In [53]:
nas_path = Path('./nasdaq_2019_to_present.csv')
nas_df = pd.read_csv(nas_path)
nas_df['date'] = pd.to_datetime(nas_df['date'])
nas_df = nas_df.set_index('date')
nas_df.columns = ['close', 'open', 'high', 'low']
nas_df

,close,open,high,low
date,,,,
2024-01-21,17462.75,17462.75,17462.75,17462.75
2024-01-20,17462.75,17462.75,17462.75,17462.75
2024-01-19,17438.50,17116.00,17471.25,17107.25
2024-01-18,17110.00,16854.50,17125.75,16834.25
2024-01-17,16869.75,16978.00,16982.00,16689.25
...,...,...,...,...
2020-07-07,10532.25,10600.00,10694.50,10505.75
2020-07-06,10598.50,10347.25,10614.50,10341.50
2020-07-03,10328.88,10348.00,10389.25,10310.20


In [54]:
length = len(nas_df)

In [55]:
end_date = datetime.now()
start_date = end_date - timedelta(days = 1825) # 60 Days is for 5 min data
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')
#start = start_date_str, end = end_date_str

In [56]:
security_df = yf.Ticker('NQ=F')
security_df.info
hist = security_df.history(start = start_date_str, end =  end_date_str, interval = '1D') # start = '2023-11-23', end = '2024-01-21'
                                                                 #Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
nasdaq_df = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']])
df_cumulative_delta = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']])
nasdaq_df


,Volume,Close,High,Low
Date,,,,
2019-01-23 00:00:00-05:00,515044,6665.25,6724.25,6592.25
2019-01-24 00:00:00-05:00,449017,6675.00,6718.50,6653.00
2019-01-25 00:00:00-05:00,446822,6791.25,6810.50,6676.75
2019-01-28 00:00:00-05:00,490045,6701.75,6791.00,6648.25
2019-01-29 00:00:00-05:00,441622,6638.75,6721.00,6616.75
...,...,...,...,...
2024-01-15 00:00:00-05:00,593445,16970.00,17024.25,16936.25
2024-01-16 00:00:00-05:00,816021,16966.50,17033.75,16811.75
2024-01-17 00:00:00-05:00,722784,16869.75,16982.00,16689.25


In [57]:
#Calcualting HLC3 value
nasdaq_df['hlc3'] = (nasdaq_df['High'] + nasdaq_df['Low'] + nasdaq_df['Close']) / 3
# Calculating VWAP (using HLC3)
nasdaq_df['vwap'] = (nasdaq_df['hlc3'] * nasdaq_df['Volume']).cumsum() / nasdaq_df['Volume'].cumsum()
#Calculating Standard Deviation
nasdaq_df['price_deviation'] = (nasdaq_df['hlc3'] - nasdaq_df['vwap']).rolling(window = 78).std() ##Window 78 set based on 5 min trading data (78 5 min in 1 day)##
# Session based (How it was on Pinescript) appears to be based on daily
# Calculate bands [UpperBand_1, LowerBand_1, UpperBand_2, LowerBand_2, UpperBand_3, LowerBand_3]
for i in [1, 2, 3]:
    nasdaq_df[f'upper_{i}'] = nasdaq_df['vwap'] + (i * nasdaq_df['price_deviation'])
    nasdaq_df[f'lower_{i}'] = nasdaq_df['vwap'] - (i * nasdaq_df['price_deviation'])
nasdaq_df = nasdaq_df.dropna()
nas_vwap = nasdaq_df
nas_vwap.columns = ['volume', 'close', 'high', 'low', 'hlc3', 'vwap', 'price_deviation', 'upper_1', 'lower_1', 'upper_2', 'lower_2', 'upper_3', 'lower_3']
nas_vwap

,volume,close,high,low,hlc3,vwap,price_deviation,upper_1,lower_1,upper_2,lower_2,upper_3,lower_3
Date,,,,,,,,,,,,,
2019-05-14 00:00:00-04:00,547769,7421.00,7459.50,7290.00,7390.166667,7325.882796,173.655504,7499.538300,7152.227292,7673.193804,6978.571788,7846.849309,6804.916284
2019-05-15 00:00:00-04:00,535915,7529.25,7538.50,7353.75,7473.833333,7328.144559,171.243996,7499.388555,7156.900563,7670.632551,6985.656567,7841.876547,6814.412571
2019-05-16 00:00:00-04:00,525588,7600.25,7641.00,7473.50,7571.583333,7331.740453,168.286170,7500.026623,7163.454283,7668.312793,6995.168113,7836.598963,6826.881943
2019-05-17 00:00:00-04:00,626777,7511.50,7620.00,7506.50,7546.000000,7335.449321,166.404533,7501.853854,7169.044788,7668.258387,7002.640256,7834.662920,6836.235723
2019-05-20 00:00:00-04:00,634294,7391.75,7559.25,7361.50,7437.500000,7337.206244,164.701080,7501.907324,7172.505164,7666.608403,7007.804085,7831.309483,6843.103005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-15 00:00:00-05:00,593445,16970.00,17024.25,16936.25,16976.833333,12187.970022,767.447409,12955.417431,11420.522612,13722.864840,10653.075203,14490.312250,9885.627794
2024-01-16 00:00:00-05:00,816021,16966.50,17033.75,16811.75,16937.333333,12193.576902,769.949111,12963.526012,11423.627791,13733.475123,10653.678680,14503.424234,9883.729569
2024-01-17 00:00:00-05:00,722784,16869.75,16982.00,16689.25,16847.000000,12198.437746,769.139256,12967.577002,11429.298490,13736.716259,10660.159233,14505.855515,9891.019977


In [58]:
def find_levels( 
        price: np.array, atr: float, # Log closing price, and log atr 
        first_w: float = 0.1, 
        atr_mult: float = 3.0, 
        prom_thresh: float = 0.1
):

    # Setup weights
    last_w = 1.0
    w_step = (last_w - first_w) / len(price)
    weights = first_w + np.arange(len(price)) * w_step
    weights[weights < 0] = 0.0

    # Get kernel of price. 
    kernal = scipy.stats.gaussian_kde(price, bw_method=atr*atr_mult, weights=weights)

    # Construct market profile
    min_v = np.min(price)
    max_v = np.max(price)
    step = (max_v - min_v) / 200
    price_range = np.arange(min_v, max_v, step)
    pdf = kernal(price_range) # Market profile

    # Find significant peaks in the market profile
    pdf_max = np.max(pdf)
    prom_min = pdf_max * prom_thresh

    peaks, props = scipy.signal.find_peaks(pdf, prominence=prom_min)
    levels = [] 
    for peak in peaks:
        levels.append(np.exp(price_range[peak]))

    return levels, peaks, props, price_range, pdf, weights


def support_resistance_levels(
        data, lookback: int, 
        first_w: float = 0.03, atr_mult: float = 3.5, prom_thresh: float = 0.20
):
    # Check if necessary columns are present in the DataFrame
    if 'high' not in data.columns or 'low' not in data.columns or 'close' not in data.columns:
        raise KeyError("Columns 'high', 'low', and 'close' must be present in the DataFrame.")

    # Get log average true range
    atr = ta.atr(np.log(data['high']), np.log(data['low']), np.log(data['close']), lookback)

    all_levels = [None] * len(data)
    for i in range(lookback, len(data)):
        i_start = i - lookback
        vals = np.log(data.iloc[i_start + 1: i + 1]['close'].to_numpy())
        levels, peaks, props, price_range, pdf, weights = find_levels(vals, atr.iloc[i], first_w, atr_mult, prom_thresh)
        all_levels[i] = levels

    return all_levels


def sr_penetration_signal(data, levels: list):
    signal = np.zeros(len(data))
    curr_sig = 0.0
    close_arr = data['close'].to_numpy()
    for i in range(1, len(data)):
        if levels[i] is None:
            continue

        last_c = close_arr[i - 1]
        curr_c = close_arr[i]

        
        for level in levels[i]:
            if curr_c > level and last_c <= level: # Close cross above line
                curr_sig = 1.0
            elif curr_c < level and last_c >= level: # Close cross below line
                curr_sig = -1.0

        signal[i] = curr_sig
    return signal

def get_trades_from_signal(data, signal: np.array):
    long_trades = []
    short_trades = []
    signals = []

    close_arr = data['close'].to_numpy()
    last_sig = 0.0
    open_trade = None
    open_signal = None
    idx = data.index
    for i in range(len(data)):
        if signal[i] == 1.0 and last_sig != 1.0:  # short entry
            if open_trade is not None:
                open_trade[2] = idx[i]
                open_trade[3] = close_arr[i]
                short_trades.append(open_trade)

            open_trade = [idx[i], close_arr[i], -1, np.nan]

        elif signal[i] == 1.0 and last_sig != 1.0:
            if open_signal is not None:
                open_signal[2] = idx[i]
                open_signal[3] = close_arr[i]
                signals.append(open_signal.copy())  # Use copy to avoid modifying the original list
                signals.append([2])  # Append the signal 2

            open_signal = [idx[i], close_arr[i], -1, np.nan]

        elif signal[i] == -1.0 and last_sig != -1.0:  # long entry
            if open_trade is not None:
                open_trade[2] = idx[i]
                open_trade[3] = close_arr[i]
                long_trades.append(open_trade)

            open_trade = [idx[i], close_arr[i], -1, np.nan]

        elif signal[i] == -1.0 and last_sig != -1.0:
            if open_signal is not None:
                open_signal[2] = idx[i]
                open_signal[3] = close_arr[i]
                signals.append(open_signal.copy())  # Use copy to avoid modifying the original list
                signals.append([1])  # Append the signal 1

            open_signal = [idx[i], close_arr[i], -1, np.nan]

        else:
            if open_signal is not None:
                open_signal[2] = idx[i]
                open_signal[3] = close_arr[i]
                signals.append(open_signal.copy())  # Use copy to avoid modifying the original list
                signals.append([0])  # Append the signal 0

            open_signal = [idx[i], close_arr[i], -1, np.nan]

    long_trades = pd.DataFrame(long_trades, columns=['exit_time', 'entry_price', 'entry_time', 'exit_price'])
    short_trades = pd.DataFrame(short_trades, columns=['exit_time', 'entry_price', 'entry_time', 'exit_price'])
    signals_df = pd.DataFrame(signals, columns=['exit_time', 'entry_price', 'entry_time', 'exit_price'])

    # Add percent column to long trades
    long_trades['percent'] = (long_trades['exit_price'] - long_trades['entry_price']) / long_trades['entry_price']

    # Add percent column to short trades
    short_trades['percent'] = -1 * (short_trades['exit_price'] - short_trades['entry_price']) / short_trades[
        'entry_price']

    # Set index for long and short trades
    long_trades = long_trades.set_index('entry_time')
    short_trades = short_trades.set_index('entry_time')

    return long_trades, short_trades, signals_df



In [59]:

# just support and resistance strategy
if __name__ == '__main__':
   
    # Trend following strategy
    data = nas_df
    plt.style.use('dark_background') 
    levels = support_resistance_levels(data, 365, first_w=1.0, atr_mult=3.0)

    data['sr_signal'] = sr_penetration_signal(data, levels)
    data['log_ret'] = np.log(data['close']).diff().shift(-1)
    data['sr_return'] = data['sr_signal'] * data['log_ret']

    long_trades, short_trades, signals_df = get_trades_from_signal(data, data['sr_signal'].to_numpy())


In [60]:
signals_df = signals_df.dropna()
signals_df

,exit_time,entry_price,entry_time,exit_price
0,2024-01-21 00:00:00,17462.75,2024-01-20,17462.75
2,2024-01-20 00:00:00,17462.75,2024-01-19,17438.50
4,2024-01-19 00:00:00,17438.50,2024-01-18,17110.00
6,2024-01-18 00:00:00,17110.00,2024-01-17,16869.75
8,2024-01-17 00:00:00,16869.75,2024-01-16,16966.50
...,...,...,...,...
726,2022-09-29 00:00:00,11228.25,2022-09-28,11555.75
728,2022-09-28 00:00:00,11555.75,2022-09-27,11333.75
730,2022-09-27 00:00:00,11333.75,2022-09-26,11316.25
732,2022-09-26 00:00:00,11316.25,2022-09-23,11376.75


In [61]:
long_trades['total_points'] = long_trades['exit_price'] - long_trades['entry_price']
long_trades['pnl_1_contract'] = long_trades['total_points'] * 20
long_trades

,exit_time,entry_price,exit_price,percent,total_points,pnl_1_contract
entry_time,,,,,,
2022-08-09,2022-08-10,13392.00,13031.50,-0.026919,-360.50,-7210.0
2022-08-08,2022-08-09,13031.50,13183.25,0.011645,151.75,3035.0
2022-08-02,2022-08-03,13271.50,12924.50,-0.026146,-347.00,-6940.0
2022-08-01,2022-08-02,12924.50,12962.50,0.002940,38.00,760.0
2022-07-29,2022-08-01,12962.50,12971.50,0.000694,9.00,180.0
...,...,...,...,...,...,...
2020-07-07,2020-07-08,10662.25,10532.25,-0.012193,-130.00,-2600.0
2020-07-06,2020-07-07,10532.25,10598.50,0.006290,66.25,1325.0
2020-07-03,2020-07-06,10598.50,10328.88,-0.025439,-269.62,-5392.4


In [62]:
long_pnl = long_trades['pnl_1_contract'].sum()
long_pnl

-194715.0

In [63]:
short_trades['total_points'] = short_trades['entry_price'] - short_trades['exit_price']
short_trades['pnl_1_contract'] = short_trades['total_points'] * 20
short_trades

,exit_time,entry_price,exit_price,percent,total_points,pnl_1_contract
entry_time,,,,,,
2022-09-20,2022-09-21,11710.00,11922.25,-0.018126,-212.25,-4245.0
2022-09-19,2022-09-20,11922.25,12024.00,-0.008534,-101.75,-2035.0
2022-09-16,2022-09-19,12024.00,11814.58,0.017417,209.42,4188.4
2022-09-15,2022-09-16,11814.58,11934.75,-0.010171,-120.17,-2403.4
2022-09-14,2022-09-15,11934.75,12142.00,-0.017365,-207.25,-4145.0
...,...,...,...,...,...,...
2021-04-16,2021-04-19,13897.25,14029.50,-0.009516,-132.25,-2645.0
2021-04-15,2021-04-16,14029.50,14014.00,0.001105,15.50,310.0
2021-04-13,2021-04-14,13798.75,13975.75,-0.012827,-177.00,-3540.0


In [64]:
short_pnl = short_trades['pnl_1_contract'].sum()
short_pnl

-165890.0